<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Recherche de régions codantes sur une phase

Dans ce complément, nous allons écrire en python l'algorithme décrit dans la vidéo, qui recherche une région codante dans un brin d'ADN. Dans cette première version nous nous concentrons sur une seule phase. Souvenez-vous qu'on peut voir trois phases sur un brin d'ADN, selon que l'on regroupe les bases en triplets en commençant à l'indice 0, 1 ou 2.

Mais comme toujours, commençons par les formules magiques

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Recherche dans l'ADN ou dans l'ARN

Dans les vidéos, on a vu les valeurs des codons **Start** et **Stop** comme contenant des `U` - et pas de `T`. Remarquons en préambule qu'il est bien sûr équivalent de chercher `AUG` dans un ARN ou `ATG` dans l'ADN correspondant.

Aussi pour des raisons pratiques, puisque les brins de matériel génétique dont nous partons sont presque toujours de l'ADN, nous allons, dans tous les notebooks de cette semaine, chercher des codons contenant des `T` et non des `U`.

### Recherche de marqueurs

Comme dans la vidéo, nous supposons que nous disposons de fonctions toutes faites pour la recherche de codons **Start** ou **Stop**. Pour l'instant nous les importons d'un autre module, mais rassurez-vous, nous verrons dès la prochaine séquence comment écrire nous-mêmes ces fonctions.

In [ ]:
# on importe les fonctions next_start_codon et next_stop_codon 
# d'une séquence prochaine
from w3_s03_c2_next_codon import next_start_codon, next_stop_codon

On rappelle que&nbsp;:
 * les indices en python commencent à 0
 * le codon **Start** est `ATG`
 * les codons **Stop** sont `TAA` ou `TAG` ou `TGA`

On précise par ailleurs que ces deux fonctions&nbsp;:
  * attendent en argument un brin `dna` et un `indice` de départ,
  * et retournent, soit l'indice de la prochaine occurrence à partir (y compris) de `indice` **et sur la même phase**, ou `None` s'il n'y a plus d'occurrence au delà de `indice`
 
Ainsi par exemple&nbsp;:

In [ ]:
# on trouve bien START car on part de l'indice 0 
# et ATG se trouve à l'indice 6, donc sur la même phase
next_start_codon("CGTACGATG", 0)

In [ ]:
# par contre ici on ne trouve rien 
# car l'indice de départ ne correspond pas à la phase 
# sur laquelle se trouve ATG
next_start_codon("CGTACGATG", 1)

### L'instruction `break`

Notre code va utiliser l'instruction `break`, qui permet d'interrompre brutalement une boucle, et donc de passer à l'instruction après la boucle. Cette construction est souvent utilisée en conjonction avec une boucle **sans fin**, comme dans cet exemple&nbsp;:

In [ ]:
# une boucle apparemment sans fin
counter = 1
while True:
    # on multiplie le compteur par 2
    counter += counter
    # une fois arrivé à 100 on sort de la boucle
    if counter >= 100:
        break
    print("compteur = ", counter)
print("après la boucle")

### L'algorithme à proprement parler

Une fois tout ceci acquis, nous pouvons écrire une fonction `coding_regions_one_phase` qui travaille sur un brin d'ADN, et qui va suivre la logique décrite dans la vidéo. Notre fonction va prendre les arguments suivants&nbsp;:
  * le brin d'ADN
  * la phase, exprimée comme un entier 0, 1 ou 2
  * la taille minimale entre 2 **Stop**&nbsp;; ce dernier argument sera optionnel, lorsqu'il est omis on prendra comme valeur par défaut 300, comme dans le cours.
  
Ce qui nous donne le code suivant&nbsp;:

In [ ]:
# recherche de gènes selon l'heuristique décrite dans la vidéo
# sur une phase seulement
# avec par défaut une longueur minimale de 300
def coding_regions_one_phase(dna, phase, minimal_length=300):
    # on initialise index à la phase; avec next_start_codon
    # et next_stop_codon on reste toujours sur la même phase
    index = phase
    # les résultats sont retournés sous la forme d'une liste 
    # de couples [start_gene, stop_gene]
    genes = []
    # on calcule stop1 qui désigne pour nous le stop "de gauche"
    # à ce stade, c'est le premier STOP à partir de la phase
    stop1 = next_stop_codon(dna, index)
    # s'il n'y a pas du tout de stop dans toute la séquence
    # on a terminé
    if not stop1:
        return genes
    # la boucle principale
    while True:
        # on cherche le premier STOP à partir de stop1
        # pour trouver le stop "de droite"
        stop2 = next_stop_codon(dna, stop1 + 3)
        # s'il n'y a plus de stop, on a fini
        if not stop2:
            return genes
        # mais il faut qu'il soit assez loin
        if stop2 - stop1 < minimal_length:
            # c'est trop court, on saute ce fragment
            stop1 = stop2
            continue
        # à ce stade on a trouvé un ORF, reste à trouver le bon start
        start = next_start_codon(dna, stop1)
        # s'il n'y en a pas: c'est qu'on ne trouvera plus rien
        # et donc on a fini
        if not start:
            return genes
        if stop2 - start < minimal_length:
            # si la region est trop petite, on l'ignore
            pass
        else:
            # cette fois on a trouvé un gène, on l'ajoute dans les résultats
            genes.append( [start, stop2] )
        # on peut passer à l'ORF suivant
        stop1 = stop2

### Sur un exemple réel

Nous allons utiliser comme ADN source celui de [Bacillus subtilis](http://www.ebi.ac.uk/ena/data/view/CP010053) (clé `CP010053`), que pour des raisons de taille nous avons déjà importé&nbsp;:

In [ ]:
from samples import subtilis
print("subtilis contient {} bases".format(len(subtilis)))

In [ ]:
# calculons les gènes sur la phase 0 avec cet algorithme
genes = coding_regions_one_phase(subtilis, 0)
print("On a trouvé {} gènes sur la phase 0".format(len(genes)))

### Quelques statistiques (optionnel)

Pour ceux que cela pourrait intéresser, et qui ont quelques connaissances en python, voici quelques statistiques sur ce résultat. 

In [ ]:
# un tableau avec toutes les longueurs de gènes
array_of_lengths = [ y-x for x,y in genes ]

# la longueur totale de tous les gènes trouvés
total_length = sum ( array_of_lengths )
# la longueur moyenne des gènes
average_length = total_length / len(genes)
print('longueur moyenne des gènes', average_length)

In [ ]:
# les tailles minimale et maximale
length_min = min ( array_of_lengths )
length_max = max ( array_of_lengths )
print("min = {}, max = {}".format(length_min, length_max))

In [ ]:
# pourcentage de la région codante par rapport à la longueur totale
print("Pourcentage de région codante", total_length/len(subtilis))

##### Un histogramme des longueurs

On peut simplement représenter la répartition des longueurs des gènes trouvés de la façon suivante. À nouveau ceci vous est donné surtout pour éveiller votre curiosité, n'hésitez pas à partager vos idées pour améliorer la présentation&nbsp;:

In [ ]:
# on importe matplotlib
import matplotlib.pyplot as plt
# pour que les figures apparaissent dans le notebook
%matplotlib inline

# la taille utile pour les courbes
import pylab
pylab.rcParams['figure.figsize'] = 8., 8.

In [ ]:
# un histogramme de la répartition des longueurs 
plt.hist(array_of_lengths, bins=75)
plt.axis([300, 7700, 0, 400])
plt.show()

### Remarque de style

Signalons enfin pour les programmeurs puristes que de très nombreuses améliorations sont possibles, tant sur le style que sur les performances. On aurait pu par exemple se définir ici une classe `Gene` et retourner une liste de `Gene` plutot qu'une liste de listes&nbsp;; ou a minima utiliser des tuples plutôt que des listes. Je vous laisse ces améliorations à titre d'exercice, mais notre parti-pris pédagogique est de nous concentrer au maximum sur les algorithmes et d'utiliser python le plus simplement possible.